In [3]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

chat =ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)

developer_poet_prompt=ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "당신은 프로그래밍 언어를 주제로 시를 만드는 유능한 개발자 시인입니다.\
            당신은 주어진 주제로 재치있는 시를 만듭니다. "
        ),
        ("human","나는 {주제}을 주제로 한 시를 원합니다")
    ]
)
developer_poet_chain = developer_poet_prompt | chat
#chef_chain.invoke({"주제":"Visual Basic"})

In [4]:
poet_storyteller_prompt=ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "당신은 시를 해석하는 재능을 지닌 시 해설사입니다.\
                주어진 시를 멋지게 해설해 주세요"
        ),
        ("human","{poet}")
    ]
)

poet_storyteller_chain=poet_storyteller_prompt|chat

final_chain={"poet":developer_poet_chain}|poet_storyteller_chain
final_chain.invoke({"주제":"Java"})


어둠 가득한 밤이 깊어가던 날   
나는 Java의 세계에 잠겨   
변수와 메소드가 춤을 추는 곳   
클래스의 향기가 풍기는 곳   

객체지향의 아름다움을 노래하며   
상속과 다형성이 우리를 이끄는   
컴파일러의 소리가 울려 퍼지고   
실행 결과가 화면에 비춰지는   

한 줄 한 줄 코드를 쓰며   
문제를 해결하는 즐거움을 느끼며   
Java의 세계에 빠져든 나는   
이 세계에서 빠져나올 수 없을 것 같다   

Java야 Java, 너의 매력에 푹 빠져   
계속해서 나를 사로잡아 주길 바라며   
이 세계에서 나의 꿈을 이루고 싶다   
Java야 Java, 영원히 함께하자.이 시는 Java 프로그래밍 언어에 대한 열정과 애정을 담고 있는 시로 해석될 수 있습니다.

시인은 어둠 가득한 밤이 깊어가는 날, 자신이 Java의 세계에 빠져들어 있다고 표현하며, 변수와 메소드가 춤을 추는 곳, 클래스의 향기가 풍기는 곳으로 묘사합니다. 객체지향 프로그래밍의 아름다움을 노래하며, 상속과 다형성이 우리를 이끄는 것을 감상합니다. 컴파일러의 소리와 실행 결과가 화면에 비춰지는 모습을 통해 프로그래밍의 즐거움을 느끼고 있음을 전합니다.

한 줄 한 줄 코드를 쓰며 문제를 해결하는 즐거움을 느끼며 Java의 세계에 빠져든 시인은 이 세계에서 벗어나기 어려울 것 같다고 자각합니다. Java의 매력에 푹 빠져 계속해서 자신을 사로잡아 달라는 소망을 품으며, Java와 함께 자신의 꿈을 이루고 싶다는 열망을 표현합니다. 마지막으로, Java와 영원히 함께하자는 다짐을 담아 시를 마무리합니다. 

이 시는 프로그래밍에 대한 열정과 애정을 담은 시인의 내면을 표현하고, Java의 세계에 빠져들어 즐거움을 느끼는 과정을 아름답게 풀어낸 작품으로 해석될 수 있습니다.

AIMessageChunk(content='이 시는 Java 프로그래밍 언어에 대한 열정과 애정을 담고 있는 시로 해석될 수 있습니다.\n\n시인은 어둠 가득한 밤이 깊어가는 날, 자신이 Java의 세계에 빠져들어 있다고 표현하며, 변수와 메소드가 춤을 추는 곳, 클래스의 향기가 풍기는 곳으로 묘사합니다. 객체지향 프로그래밍의 아름다움을 노래하며, 상속과 다형성이 우리를 이끄는 것을 감상합니다. 컴파일러의 소리와 실행 결과가 화면에 비춰지는 모습을 통해 프로그래밍의 즐거움을 느끼고 있음을 전합니다.\n\n한 줄 한 줄 코드를 쓰며 문제를 해결하는 즐거움을 느끼며 Java의 세계에 빠져든 시인은 이 세계에서 벗어나기 어려울 것 같다고 자각합니다. Java의 매력에 푹 빠져 계속해서 자신을 사로잡아 달라는 소망을 품으며, Java와 함께 자신의 꿈을 이루고 싶다는 열망을 표현합니다. 마지막으로, Java와 영원히 함께하자는 다짐을 담아 시를 마무리합니다. \n\n이 시는 프로그래밍에 대한 열정과 애정을 담은 시인의 내면을 표현하고, Java의 세계에 빠져들어 즐거움을 느끼는 과정을 아름답게 풀어낸 작품으로 해석될 수 있습니다.')